In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import glob

In [2]:
pratibha_pipeline = '/home/pjaganna1/scratch/exoncov_editc_RPS2/RPS2_APO_Illumina_rmdup_out/2020_05_18_isoform_df_RPS2_APO_Illumina_rmdup_exonIntersect_DP5_nototalMatch_0.5.csv'
sailor_pipeline = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/combined_outputs_w_cov_info/RPS2-1000_S18_L002_R1_001.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted_a0_b0_e0.01.bed'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/figure_3/outputs'

In [3]:
p = pd.read_csv(pratibha_pipeline, index_col=0)
print(p.shape[0])
p.head()

123887


,0,1,3_RPS2,4_RPS2,7_RPS2,key2_RPS2,strand_RPS2,miss_base_RPS2,total_match_RPS2,total_miss_RPS2,...,key2_APO,strand_APO,miss_base_APO,total_match_APO,total_miss_APO,total_fraction_APO,total_confidence_APO,ENMUSG,genome_key,hits2
2441,chr1,861370,G,<*>,"DP=5;I16=0,5,0,0,185,6845,0,0,100,2000,0,0,83,...",chr1:861370,-,<*>,5,0,...,chr1:861370,-,<*>,11,0,0.0,0.0,ENSG00000268179,chr1:861370,none
2442,chr1,861373,G,<*>,"DP=5;I16=0,5,0,0,185,6845,0,0,100,2000,0,0,89,...",chr1:861373,-,<*>,5,0,...,chr1:861373,-,<*>,10,0,0.0,0.0,ENSG00000268179,chr1:861373,none
2443,chr1,861376,G,<*>,"DP=5;I16=0,5,0,0,185,6845,0,0,100,2000,0,0,94,...",chr1:861376,-,<*>,5,0,...,chr1:861376,-,<*>,9,0,0.0,0.0,ENSG00000268179,chr1:861376,none
2444,chr1,861389,G,<*>,"DP=7;I16=0,7,0,0,259,9583,0,0,140,2800,0,0,118...",chr1:861389,-,<*>,7,0,...,chr1:861389,-,<*>,9,0,0.0,0.0,ENSG00000268179,chr1:861389,none
2445,chr1,861390,G,<*>,"DP=7;I16=0,7,0,0,259,9583,0,0,140,2800,0,0,120...",chr1:861390,-,<*>,7,0,...,chr1:861390,-,<*>,10,0,0.0,0.0,ENSG00000268179,chr1:861390,none


In [4]:
s = pd.read_csv(sailor_pipeline, sep='\t', names=['chrom','start','end','name','score','strand'])
s['edited'] = s['score'].apply(lambda x: int(x.split(',')[0]))
s['totcov'] = s['score'].apply(lambda x: int(x.split(',')[1]))
print(s.shape[0])
s.head()

535897


,chrom,start,end,name,score,strand,edited,totcov
0,chr1,564720,564721,0.569601,"1,57",+,1,57
1,chr1,565115,565116,0.575355,"1,56",+,1,56
2,chr1,567190,567191,1.000000,"24,26",+,24,26
3,chr1,567977,567978,0.922745,"1,9",+,1,9
4,chr1,567996,567997,0.941480,"1,7",+,1,7


In [5]:
merged = pd.merge(p, s, how='right', left_on=['0', '1', 'strand_RPS2'], right_on=['chrom','start','strand'])
merged.fillna(-1, inplace=True)
merged.head()

,0,1,3_RPS2,4_RPS2,7_RPS2,key2_RPS2,strand_RPS2,miss_base_RPS2,total_match_RPS2,total_miss_RPS2,...,genome_key,hits2,chrom,start,end,name,score,strand,edited,totcov
0,chr1,2336291.0,G,"A,<*>","DP=29;I16=5,23,1,0,1034,38186,37,1369,560,1120...",chr1:2336291,-,A,28.0,1.0,...,chr1:2336291,?,chr1,2336291,2336292,0.960596,"1,5",-,1,5
1,chr1,2336402.0,G,"A,<*>","DP=24;I16=6,17,1,0,839,30743,37,1369,460,9200,...",chr1:2336402,-,A,23.0,1.0,...,chr1:2336402,?,chr1,2336402,2336403,0.950990,"1,6",-,1,6
2,chr1,2336547.0,G,"A,<*>","DP=24;I16=8,14,2,0,802,29374,74,2738,440,8800,...",chr1:2336547,-,A,22.0,2.0,...,chr1:2336547,RPS2,chr1,2336547,2336548,0.996564,"2,10",-,2,10
3,chr1,2336655.0,C,"T,<*>","DP=28;I16=14,8,0,6,814,30118,209,7421,440,8800...",chr1:2336655,+,T,22.0,6.0,...,chr1:2336655,RPS2,chr1,2336655,2336656,1.000000,"5,12",+,5,12
4,chr1,109472634.0,G,"A,<*>","DP=23;I16=8,14,1,0,789,28557,37,1369,440,8800,...",chr1:109472634,-,A,22.0,1.0,...,chr1:109472634,?,chr1,109472634,109472635,0.932065,"1,8",-,1,8


In [6]:
merged[merged['3_RPS2']==-1][['chrom','start','end','edited','totcov','strand']].to_csv(os.path.join(output_dir, 'missing_edits_in_pratibha_data.bed'), sep='\t', header=False, index=False)